## IMPORTING RELEVANT LIBRARIES

In [1]:
import re
import pandas as pd
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, Text
from sqlalchemy.ext.declarative import declarative_base

# Saved password in config file which will be gitignored
from config import pw

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

## FIRST DATASET

### Data Extraction of First Dataset

In [2]:
gb_json_file = "resources/datasource/GB_category_id.json"
category_df = pd.read_json(gb_json_file)
category_df.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


### Data Transformation of First dataset

In [3]:
# Checking lengths of rows and columns
category_df.shape

(31, 3)

In [4]:
category_df["items"].head()

0    {'kind': 'youtube#videoCategory', 'etag': '"m2...
1    {'kind': 'youtube#videoCategory', 'etag': '"m2...
2    {'kind': 'youtube#videoCategory', 'etag': '"m2...
3    {'kind': 'youtube#videoCategory', 'etag': '"m2...
4    {'kind': 'youtube#videoCategory', 'etag': '"m2...
Name: items, dtype: object

In [5]:
# Extracting 'id' and 'category_names' columns from dictionaries
id_li = [int(item['id']) for item in category_df['items']]
title_li = [item['snippet']['title'] for item in category_df['items']]

category_dict = {'category_id' : id_li,
                 'category_name' : title_li
                }

print(category_dict)

{'category_id': [1, 2, 10, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], 'category_name': ['Film & Animation', 'Autos & Vehicles', 'Music', 'Pets & Animals', 'Sports', 'Short Movies', 'Travel & Events', 'Gaming', 'Videoblogging', 'People & Blogs', 'Comedy', 'Entertainment', 'News & Politics', 'Howto & Style', 'Education', 'Science & Technology', 'Movies', 'Anime/Animation', 'Action/Adventure', 'Classics', 'Comedy', 'Documentary', 'Drama', 'Family', 'Foreign', 'Horror', 'Sci-Fi/Fantasy', 'Thriller', 'Shorts', 'Shows', 'Trailers']}


In [6]:
# Convert dictionary to dataframes
youtube_category_df = pd.DataFrame(category_dict)
youtube_category_df.head()

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [7]:
# Checking datatypes of all columns
youtube_category_df.dtypes

category_id       int64
category_name    object
dtype: object

## SECOND DATASET

### Data Extraction of Second Dataset

In [8]:
csv_file = "resources/datasource/GBvideos.csv"

In [9]:
gb_video_df = pd.read_csv(csv_file)
gb_video_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


### Data Transformation of Second Dataset

In [10]:
gb_video_df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [11]:
# Extract relevant columns
gb_video_new_df = gb_video_df[["video_id","trending_date","category_id","channel_title","title","views",\
                               "likes","dislikes","tags","comment_count", "description"]].copy()
gb_video_new_df.head()

,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
0,Jw1Y-zhQURU,17.14.11,26,John Lewis,John Lewis Christmas Ad 2017 - #MozTheMonster,7224515,55681,10247,"christmas|""john lewis christmas""|""john lewis""|...",9479,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,24,Saturday Night Live,Taylor Swift: …Ready for It? (Live) - SNL,1053632,25561,2294,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2757,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,10,EminemVEVO,Eminem - Walk On Water (Audio) ft. Beyoncé,17158579,787420,43420,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",125882,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,17,Salford City Football Club,Goals from Salford City vs Class of 92 and Fri...,27833,193,12,"Salford City FC|""Salford City""|""Salford""|""Clas...",37,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,25,Cute Girl Videos,Dashcam captures truck's near miss with child ...,9815,30,2,[none],30,Dashcam captures truck's near miss with child ...


In [12]:
# Replace pipes in the columns using regex
pipe_to_comma = lambda val: re.sub("\|", ',', val)
pipe_to_dash = lambda val: re.sub("\|", '-', val)

gb_video_new_df["tags"] = gb_video_new_df["tags"].apply(pipe_to_comma)
gb_video_new_df["title"] = gb_video_new_df["title"].apply(pipe_to_dash)
gb_video_new_df.head()

,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
0,Jw1Y-zhQURU,17.14.11,26,John Lewis,John Lewis Christmas Ad 2017 - #MozTheMonster,7224515,55681,10247,"christmas,""john lewis christmas"",""john lewis"",...",9479,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,24,Saturday Night Live,Taylor Swift: …Ready for It? (Live) - SNL,1053632,25561,2294,"SNL,""Saturday Night Live"",""SNL Season 43"",""Epi...",2757,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,10,EminemVEVO,Eminem - Walk On Water (Audio) ft. Beyoncé,17158579,787420,43420,"Eminem,""Walk"",""On"",""Water"",""Aftermath/Shady/In...",125882,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,17,Salford City Football Club,Goals from Salford City vs Class of 92 and Fri...,27833,193,12,"Salford City FC,""Salford City"",""Salford"",""Clas...",37,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,25,Cute Girl Videos,Dashcam captures truck's near miss with child ...,9815,30,2,[none],30,Dashcam captures truck's near miss with child ...


In [13]:
# Check and ensure that all primary key vs foreign keys exist in both dataframes to ensure there is a connection
category_id = category_dict['category_id']
videodf_category_id = gb_video_new_df['category_id'].unique()
print(f"'category_id's in youtube_category_df \n -- {category_id}")
print(f"'category_id's in gb_video_new_df \n -- {videodf_category_id}")

# Compare and flag out all missing 'category_id's that will cause errors during data population
missing_ids = [mi for mi in videodf_category_id if mi not in category_id]
print(f"Flagged 'category_id's in gb_video_new_df \n -- {missing_ids}")

gb_video_new_df = gb_video_new_df.drop(gb_video_new_df[gb_video_new_df["category_id"].isin(missing_ids)].index)
print("\n")
print(f"Missing rows in dataframe shows that rows were dropped successfully")
gb_video_new_df.loc[gb_video_new_df["category_id"].isin(missing_ids)]

'category_id's in youtube_category_df 
 -- [1, 2, 10, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
'category_id's in gb_video_new_df 
 -- [26 24 10 17 25 22 23 28 15 27  1 20  2 19 29 43]
Flagged 'category_id's in gb_video_new_df 
 -- [29]


Missing rows in dataframe shows that rows were dropped successfully


,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description


In [14]:
# Checking for duplicate entries using 'video_id' values
print(gb_video_new_df.shape)
gb_video_new_df.loc[gb_video_new_df["video_id"] == "-xt_j2aIAkg"]

(38826, 11)


,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
1812,-xt_j2aIAkg,17.23.11,17,adidas Football,Predator Is Back,1624478,6674,1567,"football,""soccer"",""fussball"",""futebol"",""footba...",577,Savage Control presented by Paul Pogba.\nExplo...
2009,-xt_j2aIAkg,17.24.11,17,adidas Football,Predator Is Back,1807642,7313,1602,"football,""soccer"",""fussball"",""futebol"",""footba...",666,Savage Control presented by Paul Pogba.\nExplo...
2211,-xt_j2aIAkg,17.25.11,17,adidas Football,Predator Is Back,1875733,7896,1629,"football,""soccer"",""fussball"",""futebol"",""footba...",784,Savage Control presented by Paul Pogba.\nExplo...
2415,-xt_j2aIAkg,17.26.11,17,adidas Football,Predator Is Back,2150064,8369,1656,"football,""soccer"",""fussball"",""futebol"",""footba...",828,Savage Control presented by Paul Pogba.\nExplo...
2620,-xt_j2aIAkg,17.27.11,17,adidas Football,Predator Is Back,2318522,8722,1686,"football,""soccer"",""fussball"",""futebol"",""footba...",871,Savage Control presented by Paul Pogba.\nExplo...
2861,-xt_j2aIAkg,17.28.11,17,adidas Football,Predator Is Back,2509299,8977,1718,"football,""soccer"",""fussball"",""futebol"",""footba...",899,Savage Control presented by Paul Pogba.\nExplo...
3087,-xt_j2aIAkg,17.29.11,17,adidas Football,Predator Is Back,2678479,9250,1755,"football,""soccer"",""fussball"",""futebol"",""footba...",921,Savage Control presented by Paul Pogba.\nExplo...
3314,-xt_j2aIAkg,17.30.11,17,adidas Football,Predator Is Back,2861981,9658,1875,"football,""soccer"",""fussball"",""futebol"",""footba...",949,Savage Control presented by Paul Pogba.\nExplo...
3528,-xt_j2aIAkg,17.01.12,17,adidas Football,Predator Is Back,3054607,9915,1928,"football,""soccer"",""fussball"",""futebol"",""footba...",980,Savage Control presented by Paul Pogba.\nExplo...
3749,-xt_j2aIAkg,17.02.12,17,adidas Football,Predator Is Back,3320544,10304,2054,"football,""soccer"",""fussball"",""futebol"",""footba...",956,Savage Control presented by Paul Pogba.\nExplo...


In [15]:
# Since there are duplicate entries, will get unique entries based on 'video_id'
# Will start by sorting by views and likes
gb_video_new_df = gb_video_new_df.sort_values(by=['views', 'likes'], ascending=False)
gb_video_new_df.head()

,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
28412,_I_D_8Z4sJE,18.07.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,424538912,2818771,149275,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",99380,Disponible en tu plataforma preferida:\nhttps:...
28212,_I_D_8Z4sJE,18.06.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,413586699,2768654,145643,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",97218,Disponible en tu plataforma preferida:\nhttps:...
28008,_I_D_8Z4sJE,18.05.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,402650804,2719005,142006,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",95713,Disponible en tu plataforma preferida:\nhttps:...
27811,_I_D_8Z4sJE,18.04.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,392036878,2670723,138334,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",93834,Disponible en tu plataforma preferida:\nhttps:...
27615,_I_D_8Z4sJE,18.03.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,382401497,2618320,134625,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",92387,Disponible en tu plataforma preferida:\nhttps:...


In [16]:
# Dropping duplicates keeping the first rows
gb_video_new_df = gb_video_new_df.drop_duplicates(subset='video_id', keep="first")
gb_video_new_df.loc[gb_video_new_df["video_id"] == "-xt_j2aIAkg"]

,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
4392,-xt_j2aIAkg,17.05.12,17,adidas Football,Predator Is Back,4288382,10615,2102,"football,""soccer"",""fussball"",""futebol"",""footba...",986,Savage Control presented by Paul Pogba.\nExplo...


In [17]:
# Checking if duplicate drops are successfull by checking count
print(gb_video_new_df.count())

video_id         3266
trending_date    3266
category_id      3266
channel_title    3266
title            3266
views            3266
likes            3266
dislikes         3266
tags             3266
comment_count    3266
description      3214
dtype: int64


In [18]:
# Checking datatypes of all columns
gb_video_new_df.dtypes

video_id         object
trending_date    object
category_id       int64
channel_title    object
title            object
views             int64
likes             int64
dislikes          int64
tags             object
comment_count     int64
description      object
dtype: object

## LOADING DATA

## Connect to Local Database

In [19]:
protocol = 'postgresql'
username = 'postgres'
password = pw
host = 'localhost'
port = 5432
database_name = 'youtube_etl1'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

Base = declarative_base()

## Create database table

In [20]:
# Check for existing tables before creation
engine.table_names()

[]

In [21]:
gb_video_new_df.columns

Index(['video_id', 'trending_date', 'category_id', 'channel_title', 'title',
       'views', 'likes', 'dislikes', 'tags', 'comment_count', 'description'],
      dtype='object')

In [22]:
youtube_category_df.dtypes

category_id       int64
category_name    object
dtype: object

In [23]:
# Creating gb_video_new_df table
class gbvideo_categories(Base):
    __tablename__ = "gbvideo_categories"

    category_id = Column("category_id", Integer, primary_key = True)
    category_name = Column("category_name", String)
    
    
# Creating gb_video_new_df table
class gbvideos(Base):
    __tablename__ = "gbvideos"

    video_id = Column("video_id", String, primary_key = True)
    trending_date = Column("trending_date", String)
    category_id = Column("category_id", Integer, ForeignKey("gbvideo_categories.category_id"))
    channel_title = Column("channel_title", String)
    title = Column("title", Text)
    views = Column("views", Integer)
    likes = Column("likes", Integer)
    dislikes = Column("dislikes", Integer)
    tags = Column("tags", Text)
    comment_count = Column("comment_count", Integer)
    description = Column("description", Text)

In [24]:
Base.metadata.create_all(bind = engine)

In [25]:
# Checking for existing tables after creation
engine.table_names()

['gbvideo_categories', 'gbvideos']

In [26]:
# Populating gbvideo_categories tables with dataframe data
# First populate gbvideo_categories data as gbvideos depends on it by the due to presence of 'foreign key' constraints
youtube_category_df.to_sql(name='gbvideo_categories', con=engine, if_exists='append', index=False)

In [27]:
# Populating gbvideos tables with dataframe data
gb_video_new_df.to_sql(name='gbvideos', con=engine, if_exists='append', index=False)

### Confirm that data has been added by querying both tables

In [28]:
pd.read_sql_query('select * from gbvideo_categories', con=engine).head()

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [29]:
pd.read_sql_query('select * from gbvideos', con=engine).head()

,video_id,trending_date,category_id,channel_title,title,views,likes,dislikes,tags,comment_count,description
0,_I_D_8Z4sJE,18.07.04,10,NickyJamTV,Nicky Jam x J. Balvin - X (EQUIS) - Video Ofic...,424538912,2818771,149275,"Bad Bunny,""Amorfoda"",""Dura"",""Daddy Yankee"",""Oz...",99380,Disponible en tu plataforma preferida:\nhttps:...
1,9jI-z9QN6g8,18.18.05,10,Flow La Movie,"Te Bote Remix - Casper, Nio García, Darell, Ni...",337621571,2581961,166549,"Te Bote,""Te Bote Remix"",""Flow La Movie"",""Ozuna...",113564,Flow La Movie Presenta - Te Boté Remix - Caspe...
2,kLpH1nSLJSs,18.23.03,10,Bad Bunny,Bad Bunny - Amorfoda - Video Oficial,328860380,3823879,215530,"Bad,""Bunny"",""Bad Bunny"",""Video Oficial"",""Music...",225216,Bad Bunny - Amorfoda | Video Oficial\n\n-Click...
3,wfWkmURBNv8,18.01.03,10,Ozuna,Ozuna x Romeo Santos - El Farsante Remix,288811992,1618180,151147,"Ozuna,""Reggaeton"",""Odisea"",""Odisea The Album"",...",67506,DESCARGA Odisea Aqui: http://apple.co/2vX00sT\...
4,VYOjWnS4cMY,18.11.06,10,ChildishGambinoVEVO,Childish Gambino - This Is America (Official V...,259721696,5444541,379862,"Childish Gambino,""Rap"",""This Is America"",""mcDJ...",553371,“This is America” by Childish Gambino http://s...
